# Imports

In [ ]:
!pip install -U tqdm gdown

In [10]:
import os
import random
import zipfile

import gdown
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.models import ResNet50_Weights
from tqdm.auto import tqdm

print('Dependencies loaded.')
print("===================================================")

if(torch.cuda.is_available()):
    device = torch.device("cuda")
    print('Cuda available: {}'.format(torch.cuda.is_available()))
    print("GPU: " + torch.cuda.get_device_name(torch.cuda.current_device()))
    print("Total memory: {:.1f} GB".format((float(torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)))))
else:
    device = torch.device("cpu")
    print('Cuda not available, so using CPU. Please consider switching to a GPU runtime before running the notebook!')
    
print("===================================================")
print(F"Torch version: {torch.__version__}")
print(F"Torchvision version: {torchvision.__version__}")
print("===================================================")

Dependencies loaded.
Cuda available: True
GPU: NVIDIA GeForce RTX 3060 Laptop GPU
Total memory: 6.0 GB
Torch version: 2.0.1+cu118
Torchvision version: 0.15.2+cu118


# Dataset initialization

In [ ]:
# Dataset Download
ID = '1wYzBpucuU_RPMebUrWi2TH3Mfz-ykBGl' # PWC-Net | Real & Face2Face
output = 'Dataset.zip'

datasetDirectory = "Dataset"
if not (os.path.exists(datasetDirectory)):
  os.makedirs(datasetDirectory)

dataset_url = 'https://drive.google.com/uc?id=' + ID + '&export=download&confirm=t'
gdown.download(dataset_url, output, quiet=False)

with zipfile.ZipFile(output) as zf:
    for member in tqdm(zf.infolist(), desc='Extracting '):
        try:
            zf.extract(member, datasetDirectory)
        except zipfile.error as e:
            pass

print('Deleting the previous downloaded zip.')
os.remove('Dataset.zip')
print('Done.')

# Model (ResNet50)

In [11]:
model = models.resnet50(weights=ResNet50_Weights.DEFAULT)

# Replace the last fully connected layer
num_features = model.fc.in_features
model.fc = nn.Sequential(nn.Linear(num_features, 1, device=device),
                         nn.Sigmoid())

# Freeze pre-trained layers
for n, p in model.named_parameters():
    if(n.startswith("layer4") or n.startswith("fc")):
        p.requires_grad = True
    else:
        p.requires_grad = False
    #print(F"Parameter: {n} | Requires grad: {p.requires_grad}")

model = model.to(device)

# Helper functions

In [14]:
def loadModel(modelPath, isCheckpoint=True):
    bestAcc = 0.0
    startEpoch = 1

    if(isCheckpoint):
        print("You specified a pre-loading directory for a checkpoint.")
    else:
        print("You specified a pre-loading directory for a model.")

    if os.path.isfile(modelPath):
        print("=> Loading model '{}'".format(modelPath))
        modelLoaded = torch.load(modelPath)

        # State & Optimizer
        model.load_state_dict(modelLoaded["state_dict"], strict=False)

        # Best validations
        try:
            bestAcc = modelLoaded["best_val"]
            print(F"\t- Best validation accuracy: {bestAcc:.5f}")
        except:
            print("\t- No best validation accuracy present in this model")
            pass

        if(isCheckpoint):
            optimizer.load_state_dict(modelLoaded["optimizer"])

            # Starting epoch
            try:
                startEpoch = modelLoaded["epoch"]
                print(F"\t- Starting from epoch n.{startEpoch}")
            except:
                print("\t- No starting epoch present in this model")
                pass

        if(isCheckpoint):
            print("Checkpoint loaded successfully.")
        else:
            print("model loaded successfully.")
    else:
        if(isCheckpoint):
            print("=> No checkpoint found at '{}'".format(modelPath))
        else:
            print("=> No model found at '{}'".format(modelPath))

        print("Are you sure the directory / model exist? Exiting..")
        exit(0)

    print("===================================================")
    return bestAcc, startEpoch

def saveModel(state, is_best):
    if is_best:
        path = str(savePath) + '/best/' + F'resnet50_best_epoch-{state["epoch"]}_accT-{state["train_val"]:.5f}_accV-{state["best_val"]:.5f}.pt'
    else:
        path = str(savePath) + '/checkpoint/' + F'resnet50_checkpoint_epoch-{state["epoch"]}_accT-{state["train_val"]:.5f}_accV-{state["best_val"]:.5f}.pt'
    torch.save(state, path)

# Training

## Define parameters of the training phase

In [15]:
criterion = nn.BCELoss()  # Cross-entropy loss for classification
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)  # Optimizer for training the last layer
epochs = 50
batchSize = 64
useFullDataset = False
# ============================================== #
savePath = os.path.join("save_path", "resnet50")
if not(os.path.exists(savePath)):
    os.makedirs(savePath)

## Define the Dataloaders

In [ ]:
# Define data transformations
transformTrain = transforms.Compose([
    transforms.RandomHorizontalFlip(0.33),
    transforms.CenterCrop(286),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transformsTest = transforms.Compose([
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define paths for train_dir and test_dir according to split perfomed on OFs dataset
train_dir = os.path.join("Dataset", "train")
test_dir = os.path.join("Dataset", "test")

# Load the ImageFolder datasets for train and test directories
train_dataset = ImageFolder(root=train_dir, transform=transformTrain)
test_dataset = ImageFolder(root=test_dir, transform=transformsTest)

if not useFullDataset:

    # Define the total number of frames required
    total_frames = 85800

    # Define the desired ratio between train and test sets
    train_ratio = 0.8  # 80% for training, 20% for testing

    # Calculate the number of frames for training and testing based on the desired ratio
    train_frames = int(total_frames * train_ratio)

    # Shuffle the indices of the train dataset
    train_indices = list(range(len(train_dataset)))
    random.shuffle(train_indices)

    # Select the desired number of frames for training
    selected_train_indices = train_indices[:train_frames]

    # Calculate the remaining number of frames for testing
    remaining_frames = total_frames - train_frames

    # Check if the remaining frames exceed the number of samples in the test dataset
    if remaining_frames > len(test_dataset):
        remaining_frames = len(test_dataset)

    # Shuffle the indices of the test dataset
    test_indices = list(range(len(test_dataset)))
    random.shuffle(test_indices)

    # Select the desired number of frames for testing
    selected_test_indices = test_indices[:remaining_frames]

    # Create subsets of the train and test datasets using the selected indices
    train_dataset = torch.utils.data.Subset(train_dataset, selected_train_indices)
    test_dataset = torch.utils.data.Subset(test_dataset, selected_test_indices)

# Create data loaders for the train and test subsets
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batchSize, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batchSize, shuffle=False)

# Check the updated number of frames in each loader
train_features, train_labels = next(iter(train_loader))
print("===================================================")
print(f"Training feature shape : {train_features.size()}")
print(f"Training labels shape  : {train_labels.size()}")
print('----------------------------------------------')
test_features, test_labels = next(iter(test_loader))
print(f"Test feature shape     : {test_features.size()}")
print(f"Test labels shape      : {test_labels.size()}")
print("===================================================")
print(f"Lenght train loader: {len(train_loader)}")
print(f"Lenght test loader: {len(test_loader)}")
print("===================================================")

# Plotting
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

# Train Features
train_image = train_features[0]
axes[0].imshow(train_image.permute(1, 2, 0))
axes[0].set_title(f"Train image | Label: {train_labels[0]}")

# Test Features
test_image = test_features[0]
axes[1].imshow(test_image.permute(1, 2, 0))
axes[1].set_title(f"Test image | Label: {test_labels[0]}")

plt.show()

## Training Loop

In [20]:
load_model = False
modelPath = '' # e.g. '/save_path/resnet50/best/resnet50_epoch-1_accT-0.61361_accV-0.56974.pt'
isCheckpoint = True

if(load_model):
  bestAcc, startEpoch = loadModel(modelPath, isCheckpoint=isCheckpoint)
else:
  bestAcc = 0.00
  startEpoch = 1

In [ ]:
print("===================================================")
print(F"The directory for saving checkpoints/models is: {savePath}")
if not(os.path.exists(os.path.join(savePath, 'checkpoint'))):
    os.makedirs(os.path.join(savePath, 'checkpoint'))

if not(os.path.exists(os.path.join(savePath, 'best'))):
    os.makedirs(os.path.join(savePath, 'best'))
print("===================================================")

trainLosses = []
trainAccuracies = []
valLosses = []
valAccuracies = []
    
for epoch in range(startEpoch, epochs+1):
    # Training loop
    model.train()
    totalTrain = 0
    correctTrain = 0

    with tqdm(total=len(train_loader), desc=F'Epoch {epoch}/{epochs} | Training') as pbar:
        for i, data in enumerate(train_loader):
            images, labels = data

            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)
            outputs = outputs.view(-1) # Needed for loss computation | Shape [64] instead of [64, 1]
            lTrain = criterion(outputs, labels.float())

            # Backward and optimize
            lTrain.backward()
            optimizer.step()

            # Accuracy
            outputs_rounded = (torch.round(outputs)).int()
            correctTrain += (outputs_rounded == labels).sum().float()
            totalTrain += labels.shape[0]

            pbar.update()

    # Evaluation
    model.eval()
    with torch.no_grad():
        correctVal = 0
        totalVal = 0
        with tqdm(total=len(test_loader), desc=F'Epoch {epoch}/{epochs} | Test') as pbar:
            for i, data in enumerate(test_loader):
                images, labels = data

                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)
                outputs = outputs.view(-1) # Needed for loss computation | Shape [64] instead of [64, 1]
                lVal = criterion(outputs, labels.float())

                # Accuracy
                outputs_rounded = (torch.round(outputs)).int()
                correctVal += (outputs_rounded == labels).sum().float()
                totalVal += labels.shape[0]

                pbar.update()
            
    # Losses computation
    train_loss_epoch = lTrain.item()
    val_loss_epoch = lVal.item()
    trainLosses.append(train_loss_epoch)
    valLosses.append(val_loss_epoch)
    
    # Accuracies computation
    train_acc_epoch = correctTrain / totalTrain
    val_acc_epoch = correctVal / totalVal
    trainAccuracies.append(train_acc_epoch.detach().cpu().numpy())
    valAccuracies.append(val_acc_epoch.detach().cpu().numpy())

    if(val_acc_epoch > bestAcc):
        isBest = True
        previousBestAcc = bestAcc
        bestAcc = val_acc_epoch
        print("\nVal Accuracy increased at epoch {}: {:.5f} --> {:.5f} |".format(epoch, previousBestAcc, bestAcc))
    else:
        isBest = False

    saveModel(state={
                    'epoch': epoch,
                    'state_dict': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'train_val': train_acc_epoch,
                    'best_val': val_acc_epoch
                }, 
                is_best=isBest)

    print('\nEpoch {}/{}:\n\tTrain Acc: {:.3f} (avg. {:.3f}) | Train Loss: {:.3f} (avg. {:.3f}) | \
        \n\tVal Acc  : {:.3f} (avg. {:.3f}) | Val Loss: {:.3f} (avg. {:.3f}) |'.format(epoch, epochs, 
                                                                                    train_acc_epoch*100,
                                                                                    np.average(trainAccuracies)*100,  
                                                                                    train_loss_epoch,
                                                                                    np.average(trainLosses),
                                                                                    val_acc_epoch*100,
                                                                                    np.average(valAccuracies)*100,
                                                                                    val_loss_epoch,
                                                                                    np.average(valLosses)))
    print("=========================================")